<a href="https://colab.research.google.com/github/KIMMOOKYONG/COLAB-NOTENOOK/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%EC%A0%95%EB%B3%B4%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 참조

In [ ]:
# https://bigdata-doctrine.tistory.com/3


# 모듈 설치

In [1]:
!pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 5.3 MB/s 
     |████████████████████████████████| 51 kB 103 kB/s 
     |████████████████████████████████| 251 kB 87.9 MB/s 


# 라이브러리 로딩

In [8]:
from pykrx import stock
from datetime import datetime

# 주식 티커와 주식 이름 데이터 수집

In [13]:
# 현재일 기준으로 코스피에 상장되어 있는 주식의 이름과 티커 수집
# 수집 기준일 설정
today = datetime.today().strftime("%Y%m%d")
# 터커 목록 수집
# 파라미터
# date - 날짜, 문자열, 예시 20220703
# market - "KOSPI", "KOSDAQ", "KONEX", "ALL"
ticker_list = stock.get_market_ticker_list(date = today, market="KOSPI")

# 종목명 저장
symbol_list = []
# 티커에 매핑되는 종목명 수집
for ticker in ticker_list:
    symbol = stock.get_market_ticker_name(ticker)
    symbol_list.append(symbol)

In [18]:
# 수집 데이터 갯수 확인(티커 숫자, 종목명 숫자)
len(ticker_list), len(symbol_list)

(941, 941)

In [19]:
# 네이버 금융 웹페이지에서 수집할 데이터의 CSS Selector 설정하기
per_selector = "#_per"
pbr_selector = "#_pbr"
# 배당수익률
dividend_yield_selector = "#_dvr"

# 크롤링 모듈 설치

In [20]:
!pip install beautifulsoup4 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# PER, PBR, 배당수익률 데이터 크롤링

In [21]:
import requests
from bs4 import BeautifulSoup

In [24]:
ticker_list[:2]

['095570', '006840']

In [27]:
# 크롤링 데이터를 저장할 리스트 변수 선언
pers = []
pbrs = []
# 배당수익률
dividend_yields = []

In [28]:
for ticker in ticker_list:
    url = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    html = requests.get(url, headers={'User-agent':'Mozilla/5.0'})
    soup = BeautifulSoup(html.text, "lxml")
    per = soup.select(per_selector)
    pbr = soup.select(pbr_selector)
    dividend_yield = soup.select(dividend_yield_selector)

    if not per:  # 리스트가 비어있을 경우 None으로 변환
        per = [None]
    for pe in per:
        if pe != None:
            per_text = pe.text
            per_text = per_text.replace(",", "")  # float변환을 위해 따옴표 제거
            pers.append(float(per_text))
        else:
            per_text = pe
            pers.append(per_text)

    if not pbr:  # 리스트가 비어있을 경우 None으로 변환
        pbr = [None]
    for pb in pbr:
        if pb != None:
            pbr_text = pb.text
            pbr_text = pbr_text.replace(",", "")  # float변환을 위해 따옴표 제거
            pbrs.append(float(pbr_text))
        else:
            pbr_text = pb
            pbrs.append(pbr_text)
        
    if not dividend_yield:  # 리스트가 비어있을 경우 None으로 변환
        dividend_yield = [None]
    for d in dividend_yield:
        if d != None:
            dividend_yield_text = d.text
            dividend_yield_text = dividend_yield_text.replace(",", "")  # float변환을 위해 따옴표 제거
            dividend_yields.append(float(dividend_yield_text))
        else:
            dividend_yield_text = d
            dividend_yields.append(dividend_yield_text)

In [26]:
# print(pers)
# print(pbrs)
# print(dividend_yields)

[3.65, None]
[0.79, 0.36]
[4.19, 1.26]


# 크롤링 데이터 데이터프레임으로 변환

In [29]:
import pandas as pd 

In [30]:
stock_df = pd.DataFrame({
    "ticker":ticker_list,
    "symbol":symbol_list,
    "per":pers,
    "pbr":pbrs,
    "dividend_yield":dividend_yields
})

In [31]:
stock_df

,ticker,symbol,per,pbr,dividend_yield
0,095570,AJ네트웍스,3.65,0.79,4.19
1,006840,AK홀딩스,NaN,0.36,1.26
2,027410,BGF,11.64,0.26,2.65
3,282330,BGF리테일,21.14,4.26,1.55
4,138930,BNK금융지주,2.54,0.24,8.21
...,...,...,...,...,...
936,005010,휴스틸,3.21,0.27,3.64
937,000540,흥국화재,2.19,0.32,NaN
938,000547,흥국화재2우B,15.37,2.25,NaN
939,000545,흥국화재우,4.63,0.68,NaN


In [32]:
# per 오름차순 (NaN 제외)
per_ascending = stock_df.sort_values(by="per", ascending=True)
per_ascending[per_ascending["per"] != None].head(20)

,ticker,symbol,per,pbr,dividend_yield
37,007700,F&F홀딩스,0.25,0.33,1.02
51,011200,HMM,1.23,0.87,2.50
483,011300,성안,1.25,0.92,NaN
631,139480,이마트,1.30,0.27,1.88
26,000215,DL우,1.36,0.23,4.88
730,002025,코오롱우,1.38,0.23,3.79
780,005810,풍산홀딩스,1.41,0.27,5.53
179,013580,계룡건설,1.48,0.32,3.14
296,001230,동국제강,1.55,0.41,3.10
768,004720,팜젠사이언스,1.58,0.59,NaN


In [33]:
# pbr 오름차순 (NaN 제외)
pbr_ascending = stock_df.sort_values(by="pbr", ascending=True)
pbr_ascending[pbr_ascending["pbr"] != None].head(20)

,ticker,symbol,per,pbr,dividend_yield
44,012630,HDC,18.39,0.15,3.78
222,005725,넥센우,22.89,0.15,3.45
487,004365,세방우,2.41,0.16,2.99
286,003830,대한화섬,5.25,0.16,0.69
867,088350,한화생명,2.12,0.17,NaN
174,012320,경동인베스트,4.34,0.17,2.19
221,005720,넥센,25.83,0.17,2.91
224,002355,넥센타이어1우B,NaN,0.18,4.12
496,021820,세원정공,9.48,0.18,0.31
71,001940,KISCO홀딩스,2.03,0.19,2.88


In [34]:
# 배당수익률 내림차순
stock_df.sort_values(by="dividend_yield", ascending=False).head(20)

,ticker,symbol,per,pbr,dividend_yield
930,298020,효성티앤씨,2.13,1.14,14.53
109,005945,NH투자증권우,3.67,0.46,11.97
108,005940,NH투자증권,3.65,0.46,11.48
204,011785,금호석유우,1.58,0.48,11.38
434,016360,삼성증권,3.63,0.51,11.29
313,082640,동양생명,3.90,0.34,10.97
258,003547,대신증권2우B,1.92,0.33,10.81
797,071055,한국금융지주우,2.12,0.45,10.78
259,003545,대신증권우,2.02,0.35,10.62
201,002990,금호건설,2.13,0.43,10.15


In [35]:
# csv로 저장
stock_df.to_csv(f"{today}_stock_data_df.csv")